In [1]:
from koalas import DataFrame
from example import chess

In [2]:
# Generate the DataFrame given the chess results, which are already in the form of records.
df = DataFrame.from_records(chess('TineBerger', 2023, 9))
df[:3]

Color Moves                                         Opponent      OpponentRating Outcome Rating TimeControl
----- -----                                         --------      -------------- ------- ------ -----------
Black ['1. e4', '1... Nf6', '2. Nc3', '2... d5',... alconcha      1576           Draw    1582   180        
White ['1. Nc3', '1... g6', '2. e4', '2... Bg7',... Matiafigueroa 1637           Win     1591   180        
White ['1. Nc3', '1... d5', '2. e4', '2... d4', ... MadMauriceXIX 1556           Draw    1590   180        

In [3]:
# Get the breakdown of how many games have been won, lost and drawn.
(
    df
        .group('Outcome')
        .apply('Count', len, 'Color')
        .select('Outcome', 'Count')
)

Outcome Count
------- -----
Draw    177  
Lose    1307 
Win     1361 

In [4]:
# Identify the most difficult players.
(
    df
        .filter('Outcome', 'Lose')
        .group('Opponent')
        .apply('Losses', len, 'Outcome')
        .select('Opponent', 'Losses')
        .sort('Losses')
        .reverse()
        [:5]
)

Opponent          Losses
--------          ------
chess_enjoyer03   4     
architecturalpain 4     
searb             3     
arcanatack        3     
alwaysplexed      3     

In [5]:
# Identify which openings are most and least successful when playing as black.
def summary(df):
    return (
        df
            .filter('Color', 'Black')
            .apply('Opening', lambda moves: ' '.join(moves[:1]), 'Moves')
            .group('Opening')
            .apply('Count', len, 'Outcome')
            .apply('Win Rate', lambda outcomes: outcomes.count('Win') / len(outcomes), 'Outcome')                
            .apply('Win Percentage', lambda rate: round(100. * rate, 2), 'Win Rate')
            .sort('Win Percentage')
            .reverse()
            .apply('Minimum', lambda count: count > 20, 'Count')
            .filter('Minimum', True)
            .select('Opening', 'Win Percentage')
    )
summary(df)

Opening Win Percentage
------- --------------
1. e3   50.0          
1. e4   48.58         
1. d4   45.6          
1. Nf3  39.13         
1. c4   36.17         

In [6]:
# Compare opening performance against another player
left = summary(df).rename('Win Percentage', 'Win Percentage (Left)')
right = summary(DataFrame.from_records(chess('architecturalpain', 2023, 9))).rename('Win Percentage', 'Win Percentage (Right)')
comparison = left.join(right, 'Opening').apply('Difference', lambda a, b: round(b - a, 2), 'Win Percentage (Left)', 'Win Percentage (Right)').export('csv', 'performance.csv')
comparison

Opening    Win Percentage (Left) Win Percentage (Right) Difference
-------    --------------------- ---------------------- ----------
('1. d4',) 45.6                  51.4                   5.8       
('1. e4',) 48.58                 50.2                   1.62      